In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools
# from test_utils import *
import test_utils
import analysis

In [2]:
Lx, Lt = 4, 4
# Lx, Lt = 2, 8
V = (Lx ** 3) * Lt
LL = [Lx, Lx, Lx, Lt]
analysis.set_dimensions(Lx, Lt)

(4, 4, 256, [4, 4, 4, 4])

In [3]:
# Load in the free field propagators
job_num = 30558
# job_num = 30562
k1 = np.array([-1, 0, 1, 0])
k2 = np.array([0, 1, 1, 0])
q = k2 - k1
q_lat = np.sin(analysis.to_linear_momentum(q + analysis.bvec))
qlat_slash = analysis.slash(q_lat)
test_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/free_field_props_' + str(job_num) + '/free_field_props.h5'
prop_k1, prop_k2, prop_q = test_utils.read_propagators(test_file, q)

In [4]:
# Fourier transform and momentum project
FTk1 = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTk2 = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTq = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTmq = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTm2q = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
for x, y, z in itertools.product(range(Lx), repeat = 3):
    for t in range(Lt):
        pos = np.array([x, y, z, t])
        FTk1[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, k1 + analysis.bvec))
        FTk2[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, k2 + analysis.bvec))
        FTq[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, q + analysis.bvec))
        FTmq[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, -q))
        FTm2q[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, -2 * q))

In [5]:
Sk1 = np.einsum('xyzt,xyzt...->...', FTk1, prop_k1) / V
Sk2 = np.einsum('xyzt,xyzt...->...', FTk2, prop_k2) / V
Sq = np.einsum('xyzt,xyzt...->...', FTq, prop_q) / V
antiprop_k2 = np.einsum('ik,...akbl,lj->...aibj', analysis.gamma5, analysis.adjoint(prop_k2), analysis.gamma5)
Sk1_inv = np.linalg.tensorinv(Sk1)
Sk2_inv = np.linalg.tensorinv(Sk2)
Sq_inv = np.linalg.tensorinv(Sq)

## Verify that Zq, ZV, and ZA are all equal to 1 for the free field

In [6]:
# Verify Zq is 1
Zq = (1j) * sum([q[mu] * np.einsum('ij,ajai', analysis.gamma[mu], Sq_inv) for mu in range(analysis.d)]) \
    / (12 * analysis.square(q))
print(Zq)    # for free field, Zq must be 1.

(1.0000000085571377-2.220446049250313e-16j)


In [7]:
# Get and amputate three point functions
GV = np.array([np.einsum('xyzt,xyztaick,kl,xyztclbj->aibj', FTmq, antiprop_k2, analysis.gamma[mu], prop_k1) for mu in range(4)]) / V
GA = np.array([np.einsum('xyzt,xyztaick,km,ml,xyztclbj->aibj', FTmq, antiprop_k2, analysis.gamma[mu], analysis.gamma5, prop_k1) for mu in range(4)]) / V
ΓV = np.array([np.einsum('aibj,bjck,ckdl->aidl', Sk2_inv, GV[mu], Sk1_inv) for mu in range(4)])
ΓA = np.array([np.einsum('aibj,bjck,ckdl->aidl', Sk2_inv, GA[mu], Sk1_inv) for mu in range(4)])
qΓV = np.einsum('m,maibj->aibj', q_lat, ΓV)
qΓA = np.einsum('m,maibj->aibj', q_lat, ΓA)

In [8]:
# Verify axial and vector R.C.s are 1
ZV = 12 * Zq * analysis.square(q_lat) / np.einsum('aiaj,ji->', qΓV, qlat_slash)
ZA = 12 * Zq * analysis.square(q_lat) / np.einsum('aiaj,jk,ki->', qΓA, analysis.gamma5, qlat_slash)
print(ZV)
print(ZA)

(0.9999999914428666-8.537942317205117e-18j)
(0.9999999914428678+1.3709333589380688e-16j)


## Play around with the propagators and find some way to get the amputated 4 point function equal to its tree level value.

In [52]:
# Confirm that I'm getting the right combinations of gamma matrices
A = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
# An_contr = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
GO = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
ΓO = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
for n in range(16):
    print(n)
    A[n] = np.einsum('...aick,kl,...clbj->...aibj', antiprop_k2, analysis.gammaList[n], prop_k1)
    An_contr = np.einsum('...ckdl,...aibj->...aibjckdl', A[n], A[n]) - np.einsum('...ckbj,...aidl->...aibjckdl', A[n], A[n])
    GO[n] = 2 * np.einsum('xyzt,xyzt...->...', FTm2q, An_contr) / V    # TODO later try with no momentum
    ΓO[n] = np.einsum('aiem,gobj,ckfn,hpdl,emgofnhp->aibjckdl', Sk2_inv, Sk1_inv, Sk2_inv, Sk1_inv, GO[n])  #z is n

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [53]:
# Use reverse gamma matrices that Phiala uses
AR = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
# An_contrR = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
GOR = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
ΓOR = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
for n in range(16):
    print(n)
    AR[n] = np.einsum('...aick,kl,...clbj->...aibj', antiprop_k2, analysis.gammaListRev[n], prop_k1)
    An_contrR = np.einsum('...ckdl,...aibj->...aibjckdl', AR[n], AR[n]) - np.einsum('...ckbj,...aidl->...aibjckdl', AR[n], AR[n])
    GOR[n] = 2 * np.einsum('xyzt,xyzt...->...', FTm2q, An_contrR) / V    # TODO later try with no momentum
    ΓOR[n] = np.einsum('aiem,gobj,ckfn,hpdl,emgofnhp->aibjckdl', Sk2_inv, Sk1_inv, Sk2_inv, Sk1_inv, GOR[n])  #z is n

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [54]:
for m in range(16):
    if np.allclose(A[m], AR[m]):
        print('n = ' + str(m) + ', +')
    elif np.allclose(A[m], -AR[m]):
        print('n = ' + str(m) + ', -')

n = 0, +
n = 1, +
n = 2, +
n = 3, -
n = 4, +
n = 5, -
n = 6, -
n = 7, -
n = 8, +
n = 9, -
n = 10, -
n = 11, -
n = 12, -
n = 13, -
n = 14, -
n = 15, +


In [55]:
for m in range(16):
    if np.allclose(GO[m], GOR[m]):
        print('n = ' + str(m) + ', +')
    elif np.allclose(GO[m], -GOR[m]):
        print('n = ' + str(m) + ', -')
    else:
        print('n = ' + str(m) + ', no match')

n = 0, +
n = 1, +
n = 2, +
n = 3, +
n = 4, +
n = 5, +
n = 6, +
n = 7, +
n = 8, +
n = 9, +
n = 10, +
n = 11, +
n = 12, +
n = 13, +
n = 14, +
n = 15, +


In [60]:
SS = ΓO[0]
PP = ΓO[15]
VV = ΓO[1] + ΓO[2] + ΓO[4] + ΓO[8]
# AA = -ΓO[14] + ΓO[13] - ΓO[11] + ΓO[7]    # SOMETHING WRONG WITH AA
# AA = -(ΓO[14] + ΓO[13] + ΓO[11] + ΓO[7])
AA = ΓO[14] + ΓO[13] + ΓO[11] + ΓO[7]
TT = (ΓO[3] + ΓO[5] + ΓO[6] + ΓO[9] + ΓO[10] + ΓO[12])

# SS = ΓOR[0]
# PP = ΓOR[15]
# VV = ΓOR[1] + ΓOR[2] + ΓOR[4] + ΓOR[8]
# AA = ΓOR[14] + ΓOR[13] + ΓOR[11] + ΓOR[7]
# TT = (ΓOR[3] + ΓOR[5] + ΓOR[6] + ΓOR[9] + ΓOR[10] + ΓOR[12])

eps = 1e-5
SS[np.abs(SS) < eps] = 0
PP[np.abs(PP) < eps] = 0
VV[np.abs(VV) < eps] = 0
AA[np.abs(AA) < eps] = 0
TT[np.abs(TT) < eps] = 0
# Gamma_D = np.einsum('aiajckcl->ijkl', Gamma)

In [123]:
# [a, i, b, j, c, k, d, l]
# SS[:, 0, :, 0, :, 1, :, 1]

In [75]:
Gamma = [VV + AA, VV - AA, SS - PP, SS + PP, TT]
P = analysis.projectors()
Lambda = np.einsum('nbjaidlck,maibjckdl->mn', P, Gamma)
F = analysis.getF()

In [58]:
print(F)

[[3072.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j 2304.+0.j -384.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j -384.+0.j  576.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j  480.+0.j  288.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j  288.+0.j 2016.+0.j]]


In [76]:
print(Lambda.astype(int))

[[3072    0    0    0    0]
 [   0 2304 -384    0    0]
 [   0 -384  576    0    0]
 [   0    0    0  480  288]
 [   0    0    0  288 2015]]


/Users/theoares/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.


In [20]:
for i in range(5):
    print(np.einsum('nbjaidlck,aibjckdl->n', P, Gamma[i]))

[ 2.975464e-04+9.7787338e-14j  2.304000e+03-4.5474735e-13j
 -3.840001e+02+0.0000000e+00j  0.000000e+00+0.0000000e+00j
  0.000000e+00+0.0000000e+00j]
[ 3.072001e+03-1.1394893e-12j  2.632141e-04+0.0000000e+00j
 -2.002716e-04+0.0000000e+00j  0.000000e+00+0.0000000e+00j
  0.000000e+00+0.0000000e+00j]
[ 0.0000000e+00+0.000000e+00j -3.8400009e+02+0.000000e+00j
  5.7600024e+02+0.000000e+00j  0.0000000e+00-7.640903e-16j
 -1.1444092e-04-7.640903e-16j]
[ 0.0000000e+00+0.000000e+00j  2.4604797e-04+0.000000e+00j
 -8.8691711e-05+0.000000e+00j  4.8000034e+02-3.121692e-15j
  2.8800012e+02-3.121692e-15j]
[   0.     +0.0000000e+00j    0.     +0.0000000e+00j
    0.     +0.0000000e+00j  288.00006-1.1668785e-14j
 2015.9998 -1.1668785e-14j]


In [139]:
print(Lambda)

[[ 3.0720010e+03-1.1394893e-12j  2.6321411e-04+0.0000000e+00j
  -2.0027161e-04+0.0000000e+00j  0.0000000e+00+0.0000000e+00j
   0.0000000e+00+0.0000000e+00j]
 [ 2.9754639e-04+9.7787338e-14j  2.3040000e+03-4.5474735e-13j
  -3.8400009e+02+0.0000000e+00j  0.0000000e+00+0.0000000e+00j
   0.0000000e+00+0.0000000e+00j]
 [ 0.0000000e+00+0.0000000e+00j -3.8400009e+02+0.0000000e+00j
   5.7600024e+02+0.0000000e+00j  0.0000000e+00-7.6409032e-16j
  -1.1444092e-04-7.6409032e-16j]
 [ 0.0000000e+00+0.0000000e+00j  2.4604797e-04+0.0000000e+00j
  -8.8691711e-05+0.0000000e+00j  4.8000034e+02-3.1216921e-15j
   2.8800012e+02-3.1216921e-15j]
 [ 0.0000000e+00+0.0000000e+00j  0.0000000e+00+0.0000000e+00j
   0.0000000e+00+0.0000000e+00j  2.8800006e+02-1.1668785e-14j
   2.0159998e+03-1.1668785e-14j]]


In [140]:
print(Lambda.astype(int))    # note this truncates so some may be 1 less than in F

[[3072    0    0    0    0]
 [   0 2304 -384    0    0]
 [   0 -384  576    0    0]
 [   0    0    0  480  288]
 [   0    0    0  288 2015]]


/Users/theoares/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.


In [76]:
F

array([[3072.+0.j,    0.+0.j,    0.+0.j,    0.+0.j,    0.+0.j],
       [   0.+0.j, 2304.+0.j, -384.+0.j,    0.+0.j,    0.+0.j],
       [   0.+0.j, -384.+0.j,  576.+0.j,    0.+0.j,    0.+0.j],
       [   0.+0.j,    0.+0.j,    0.+0.j,  480.+0.j,  288.+0.j],
       [   0.+0.j,    0.+0.j,    0.+0.j,  288.+0.j, 2016.+0.j]],
      dtype=complex64)

In [103]:
# Check out what the VV and AA amputated 4-pt functions actually look like
VV_ijkl = VV[0, :, 0, :, 1, :, 1, :]#.astype(int)
AA_ijkl = AA[0, :, 0, :, 1, :, 1, :]#.astype(int)

In [106]:
print(np.allclose(2 * np.einsum('mij,mkl->ijkl', analysis.gamma, analysis.gamma), VV_ijkl))

True
False


In [112]:
gamGam5 = np.array([analysis.gamma[mu] @ analysis.gamma5 for mu in range(4)])
print(np.allclose(2 * np.einsum('mij,mkl->ijkl', gamGam5, gamGam5), AA_ijkl))
# 2 * np.einsum('mij,mkl->ijkl', gamGam5, gamGam5) == AA_ijkl.astype(int)

False


In [108]:
2 * np.einsum('mij,mkl->ijkl', gamGam5, gamGam5)

array([[[[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]],

        [[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]],

        [[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  4.+0.j],
         [-4.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]],

        [[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j, -4.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [-4.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]]],


       [[[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]],

        [[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
 

In [109]:
AA_ijkl

array([[[[ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0000000e+00j],
         [ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0000000e+00j],
         [ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0000000e+00j],
         [ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0000000e+00j]],

        [[ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0000000e+00j],
         [ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0000000e+00j],
         [ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0000000e+00j],
         [ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0000000e+00j]],

        [[ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  4.-7.1738597e-08j,
           0.+0.0000000e+00j],
         [ 0.+0.0000000e+00j,  0.+0.0000000e+00j,  0.+0.0000000e+00j,
           0.+0.0

## Compare with QLUA output. There seems to be a bug somewhere, since the 3-point functions don't match

In [69]:
# Saving mqFT2, A_Gamma, and the Green's function. Compare all of them to what this code outputs. 
# when everything here matches, I should be able to feed QLUA output into free_field.py and have it work
qlua_jobnum = 30661
qlua_out_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/'+'free_field_'+str(qlua_jobnum)+'/free_field.h5'
fq = h5py.File(qlua_out_file, 'r')
mqFT2 = fq['mqFT2'][()]
A_gamma = np.einsum('...ijab->...aibj', np.array([fq['Agamma' + str(n)][()] for n in range(16)]))
qlua_GO = np.einsum('nijklabcd->naibjckdl', np.array([fq['Gn' + str(n) + '/q1100'][()] for n in range(16)]))

In [66]:
# Compare FT factors from QLUA with Python
# print(mqFT2)
# print(FTm2q)
print(np.allclose(mqFT2, FTm2q))

True


In [ ]:
# Compare A_gamma output from QLUA and Python for some n.
n = 3
print(A_gamma[n])
print(A[n])

In [74]:
# Compare QLUA GO output with Python GO output
# print(qlua_GO[n])
# print(GO[n])    # python output for GO at gamma = 3
print(np.allclose(qlua_GO[n], GO[n]))
print(np.allclose(np.einsum('aibjckdl->dlckbjai', qlua_GO[n]), GO[n]))

False
True
